In [1]:
#importing packages
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
#this is a bubble detecting algorithm developed by Phillips and Yu (2011) and Phillips et al. (2015) 
#it is based on the augmented Dickey-Fuller test

#downloading data
ticker = 'IOC.NS'
start = '2020-09-01'
#end = '2021-06-15'
prices = yf.download(ticker, start)['Close']
print(prices)


[*********************100%***********************]  1 of 1 completed
Date
2020-09-01    56.433334
2020-09-02    56.866665
2020-09-03    56.733334
2020-09-04    55.933334
2020-09-07    55.866665
                ...    
2023-07-04    94.650002
2023-07-05    95.500000
2023-07-06    98.599998
2023-07-07    99.199997
2023-07-10    97.400002
Name: Close, Length: 709, dtype: float64


DatetimeIndex(['2021-10-07', '2021-10-08', '2021-10-11', '2021-10-14',
               '2021-10-18'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [4]:
#inputting the parameters
r0 = int(len(prices)*0.1)
#specify lags for the augmented Dickey-Fuller (ADF) test
adf_lags = 3
#critical value of the right-tailed ADF-test (95%) from Phillips et al. (2015)
crit = 1.49
#transforming data
log_prices = np.array(np.log(prices))
delta_log_prices = log_prices[1:] - log_prices[:-1]
n = len(delta_log_prices)
BSADF = np.array([])
#calculating ADF stats
for r2 in range(r0,n):
    ADFS = np.array([])
    for r1 in range(0,r2-r0+1):
        X0 = log_prices[r1:r2+1]
        X = pd.DataFrame()
        X[0] = X0
        for j in range(1,adf_lags+1):
            X[j] = np.append(np.zeros(j),delta_log_prices[r1:r2+1-j])
        X = np.array(X)
        Y = delta_log_prices[r1:r2+1]
        reg = sm.OLS(Y,sm.add_constant(X))
        res = reg.fit()
        ADFS = np.append(ADFS, res.params[1]/res.bse[1])
    BSADF = np.append(BSADF, max(ADFS))
#visualising the results
plt.rc('xtick',labelsize = 8)
plt.plot(prices.index[r0+1:],BSADF)
plt.plot(prices.index[r0+1:],np.ones(len(BSADF))*crit)
#printing dates when bubbles were detected
print(prices.index[r0+1:][BSADF > crit])